In [1]:
# Contact info:
# Dimosthenis Beleveslis
# d.beleveslis@gmail.com

# Efood Assessment: Part II - Analyze Users

## 1) Import python libraries

In [2]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt

# Pandas display options
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

## 2) Read data

In [3]:
df = pd.read_csv('Data/orders.csv')

In [4]:
df.head()

,order_id,user_id,order_timestamp,city,cuisine,paid_cash,amount
0,7322173742136,94223136441,2022-01-18 15:41:28 UTC,Αίγιο,Breakfast,True,3.6
1,7210978616924,204794985423,2022-01-02 17:50:50 UTC,Αγρίνιο,Breakfast,True,1.9
2,7221232906132,12703920390,2022-01-04 08:37:12 UTC,Αγρίνιο,Breakfast,True,1.7
3,7399605199800,673310109528,2022-01-31 11:33:05 UTC,Λιβαδειά,Breakfast,True,3.0
4,7264221279744,673310109528,2022-01-10 11:18:28 UTC,Λιβαδειά,Breakfast,True,3.0


In [5]:
df.shape

(534270, 7)

## 3) EDA

In [6]:
# Number of (unique) orders
print('- There are', len(df.order_id), 'rows.')
print('- There are', df.order_id.nunique(), 'unique orders.')
print('-- So, each row corresponds to a unique order.')

- There are 534270 rows.
- There are 534270 unique orders.
-- So, each row corresponds to a unique order.


In [7]:
# Number of (unique) customers
print('- There are', df.user_id.nunique(), 'unique customers.')
print('-- So, there are customers that have ordered more than once.')

- There are 121943 unique customers.
-- So, there are customers that have ordered more than once.


In [8]:
# Top-10 customers based on the order frequency
df.user_id.value_counts(normalize=False).to_frame().head(10)

,user_id
154559060124,135
104460666828,107
119884693362,105
32214575190,100
247344644274,88
131345003580,87
88909070628,77
476333229792,74
17562694464,71
117577441989,71


In [9]:
# Top-10 customers based on the total order value
df.groupby('user_id').amount.sum().to_frame().sort_values(by='amount', ascending=False).head(10)

,amount
user_id,
485537911656,1351.50
154559060124,945.40
629495492640,863.05
119884693362,839.95
589547556318,796.00
581230298733,690.80
264820653468,630.10
104460666828,621.50
42248071740,611.20


In [10]:
# Time period
df.order_timestamp.min()
df.order_timestamp.max()
print('Time period:', df.order_timestamp.min(), ' - ', df.order_timestamp.max())

Time period: 2022-01-01 00:06:07 UTC  -  2022-01-31 23:59:53 UTC


In [11]:
# Basic statistics for amount (= order value)
df.amount.describe()

count    534270.000000
mean          8.522366
std           6.095029
min           0.400000
25%           4.600000
50%           6.700000
75%          10.800000
max         204.800000
Name: amount, dtype: float64

In [12]:
# Cecking if there are missing values
print(df.isnull().sum(axis=0))
print('- There are no missing values in the dataset.')

order_id           0
user_id            0
order_timestamp    0
city               0
cuisine            0
paid_cash          0
amount             0
dtype: int64
- There are no missing values in the dataset.


In [13]:
df.city.unique()

array(['Αίγιο', 'Αγρίνιο', 'Λιβαδειά', 'Λαμία', 'Βόλος', 'Λάρισα',
       'Ιωάννινα', 'Φλώρινα', 'Νάουσα', 'Δράμα', 'Ξάνθη',
       'Αλεξανδρούπολη', 'Ρόδος', 'Άργος', 'Ναύπλιο', 'Ζάκυνθος',
       'Μεσολόγγι', 'Λευκάδα', 'Θήβα', 'Άρτα', 'Γρεβενά', 'Γιαννιτσά',
       'Βέροια', 'Αλεξάνδρεια', 'Ορεστιάδα', 'Διδυμότειχο', 'Λέρος',
       'Γύθειο', 'Αμαλιάδα', 'Ναύπακτος', 'Αράχωβα', 'Έδεσσα', 'Αίγινα',
       'Νάξος', 'Αλίαρτος', 'Οινόφυτα', 'Μεγαλόπολη', 'Ιστιαία',
       'Μύκονος', 'Λεωνίδιο', 'Βασιλικό', 'Γαργαλιάνοι', 'Αριδαία',
       'Φιλιατρά', 'Ηγουμενίτσα', 'Άνδρος'], dtype=object)

# Segment existing customers based on their frequency and order value

## 4) Customer segmentation based on RFM analysis

<b>RFM</b> stands for Recency, Frequency, and Monetary value </br>
- <b>Recency</b>: How recently a customer made an order. </br>
- <b>Frequency</b>: How often customers make an order. </br>
- <b>Monetary Value</b>: How much a customer spends on orders. (We have used 'amount' for this) </br>

### 4.1) Preprocess dataset

In [14]:
# convert timestamp to datetime
df['order_timestamp_2'] = pd.to_datetime(df['order_timestamp'])
# convert datetime to date
df['Date_Order'] = df['order_timestamp_2'].dt.date

In [15]:
# As we mentioned above, the last order date is '2022-01-31'. 
# So, we assume that '2022-02-01' is the current date and we calculate recency based on that.
current_date = dt.date(2022, 2, 1)
current_date

datetime.date(2022, 2, 1)

In [16]:
# create a df with the 3 metrics (Recency, Frequency, Monetary_value) 
df_rfm = df.groupby('user_id').agg({'Date_Order': lambda x: (current_date - x.max()).days,
                                          'order_id': lambda x: len(x),
                                          'amount': lambda x: x.sum()})

# rename the columns of the dataframe
df_rfm = df_rfm.rename(columns={'Date_Order': 'Recency', 
                         'order_id': 'Frequency', 
                         'amount': 'Monetary_value'})

In [17]:
# Split the ranges of each of the 3 metrics to 4 groups
df_rfm["R"] = pd.qcut(df_rfm["Recency"],4,labels=[1,2,3,4])
df_rfm["F"] = pd.qcut(df_rfm["Frequency"],5,labels=[4,3,2,1], duplicates='drop')
df_rfm["M"] = pd.qcut(df_rfm["Monetary_value"],4,labels=[4,3,2,1])

In [18]:
# Create a column with the 3 metrics (R, F, M) merged as one.
# e.g. R=1, F=2, M=4 ---> RFM=124
df_rfm["RFM"] = df_rfm["R"].astype(str) +df_rfm["F"].astype(str) + df_rfm["M"].astype(str)

In [19]:
# Create a column with the RFM_Score which is the sum of the digits of the RFM metric
df_rfm["RFM_Score"] = df_rfm["RFM"].apply(lambda x: sum(int(i) for i in x))

# The lower the RFM_Score is, the better the customer is
df_rfm = df_rfm.sort_values(by=['RFM_Score'])

### 4.2) Segmentation based on RFM_Score

We will create 4 segments as following:
- <b>1.Passionate efooders</b>: Very loyal and worthy customers
- <b>2.Promising efooders</b>: Often ordering from efood
- <b>3.Struggling users</b>: Ordering from efood once in a while
- <b>4.Lost users</b>: Ordered from efood but have stopped

In [20]:
# Assign a segment name base on the RFM_Score
df_rfm['Segment'] = df_rfm['RFM_Score'].apply(lambda x: '1.Passionate efooders' if x<5
                                  else '2.Promising efooders' if x<7
                                  else '3.Struggling users' if x<10 # 
                                    else '4.Lost users'
                                  )

In [21]:
# print 10 random users and the Segment that they belong tp
df_rfm.sample(10)

,Recency,Frequency,Monetary_value,R,F,M,RFM,RFM_Score,Segment
user_id,,,,,,,,,
193851034776,23,2,17.7,4,4,3,443,11,4.Lost users
142328156670,3,1,4.5,1,4,4,144,9,3.Struggling users
656959326891,5,10,80.5,2,1,1,211,4,1.Passionate efooders
88910105676,1,3,16.6,1,3,3,133,7,3.Struggling users
30246431418,7,1,28.9,2,4,2,242,8,3.Struggling users
283791660117,5,5,56.9,2,2,1,221,5,2.Promising efooders
308904124074,1,9,52.3,1,1,1,111,3,1.Passionate efooders
599772277986,2,2,10.0,1,4,4,144,9,3.Struggling users
673312050243,3,13,64.0,1,1,1,111,3,1.Passionate efooders


In [22]:
# print the percentage of users that belong to each segment
df_segments = df_rfm.Segment.value_counts(normalize=True).to_frame()*100
df_segments.round(1).rename(columns={'Segment':'Perc(%)'}).sort_index()

,Perc(%)
1.Passionate efooders,19.5
2.Promising efooders,15.7
3.Struggling users,26.8
4.Lost users,38.0


## 5) Choose segment for a Marketing campaign about Breakfast that could increase loyalty

Loyalty is highly correlated with the frequency of ordering. Thus, a customer that orders often is considered to be loyal. The target group of the Marketing campaign should be a segment that contains customers that can potentially like ordering breakfast and can potentially become more loyal (increase their frequency and value).

So, the Marketing team should not target to the 'best' (1. Passionate efooders) or to the 'worst' (4.Lost users) segments. The target group should be one of the '2.Promising efooders', '3.Struggling users' or both.

Below we investigate which of the 2 above segments ('2.Promising efooders', '3.Struggling users') contain more customers that order breakfast.

In [23]:
# create a new dataframe with the segment of each unique customer
df_cust_segments = df_rfm.reset_index()[['user_id', 'Segment']]

In [24]:
# Merge the above dataframe with the initial dataframe
df_cs = df[['user_id', 'cuisine']].merge(df_cust_segments, left_on=['user_id'], right_on=['user_id'], how='left')

In [25]:
# create a dataframe with the number of orders in each segment
df_so = df_cs['Segment'].value_counts().to_frame().reset_index()
df_so = df_so.rename(columns={'index':'Segment', 'Segment':'N'})

In [26]:
# calculate the percentage of breakfast orders in each segment
df_cs = df_cs.groupby(['Segment', 'cuisine']).user_id.count().to_frame().reset_index()
df_cs = df_cs[df_cs.cuisine=='Breakfast']
df_cs = df_cs.rename(columns={'user_id':'N_Breakfast'})
df_cs = df_cs.merge(df_so, left_on=['Segment'], right_on=['Segment'], how='left')
df_cs['Perc(%)'] = (df_cs['N_Breakfast']/df_cs['N'])*100
df_cs['Perc(%)'] = df_cs['Perc(%)'].round(1)
df_cs

,Segment,cuisine,N_Breakfast,N,Perc(%)
0,1.Passionate efooders,Breakfast,132105,291292,45.4
1,2.Promising efooders,Breakfast,33689,99341,33.9
2,3.Struggling users,Breakfast,24833,83750,29.7
3,4.Lost users,Breakfast,17336,59887,28.9


- 33.9% of the orders in segment '2.Promising efooders' are from Breakfast cuisine. While the 29.7% of the orders in Segment '3.Struggling users' are from Breakfast cuisine. </br>
- Hence, if we need to select only one segmet, we would select segment '2.Promising efooders' because they are already more interested in ordering this specific type of food (breakfast).